## Modificando dados usando SQLAlchemy
***

Vamos fazer atualizações, inserções e deleções usando o SQLALchemy

In [1]:
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import sessionmaker
url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using (psycopg2)
    username='notebook',
    password='notebook',
    host='postgres',
    database='notebook',
    port=5432
)
engine = create_engine(url, pool_size=10, max_overflow=20, pool_recycle=3600)
Session = sessionmaker(bind=engine)

In [2]:
from sqlalchemy import INTEGER
from sqlalchemy.orm import DeclarativeBase


class Base(DeclarativeBase):
    pass

In [3]:
import datetime
from typing import Optional
from sqlalchemy.dialects.postgresql import TIMESTAMP
from sqlalchemy.sql.functions import func
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey, BIGINT, VARCHAR, String, DECIMAL

class TimestampMixin:
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    updated_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now(), onupdate=func.now())


class User(Base, TimestampMixin):
    """
    Classe de usuário no banco de dados
    """

    __tablename__ = "users"

    telegram_id: Mapped[int] = mapped_column(BIGINT, primary_key=True)
    full_name: Mapped[str] = mapped_column(VARCHAR(255))
    username: Mapped[Optional[str]] = mapped_column(VARCHAR(255), nullable=True)
    language_code: Mapped[str] = mapped_column(VARCHAR(255))
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    referrer_id: Mapped[Optional[int]] = mapped_column(BIGINT, ForeignKey('users.telegram_id', ondelete='SET NULL'))
    orders: Mapped[list['Order']] = relationship(back_populates='user')
    
class Order(Base, TimestampMixin):
    """
    Tabela de pedidos
    """

    __tablename__ = "orders"

    order_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    user_id: Mapped[int] = mapped_column(BIGINT, ForeignKey("users.telegram_id", ondelete="CASCADE"))
    products: Mapped[list['OrderProducts']] = relationship()
    user: Mapped['User'] = relationship(back_populates='orders')
    
class Product(Base, TimestampMixin):
    """
    Tabela de produtos
    """
    
    __tablename__ = "products"

    product_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    title: Mapped[str] = mapped_column(String(255))
    description: Mapped[str]
    price: Mapped[float] = mapped_column(DECIMAL(precision=16, scale=4))

class OrderProducts(Base):
    """
    Cria a tabela estrangeira do relacionamento entre pedido e produtos
    """
    
    __tablename__ = "order-products"

    order_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("orders.order_id", ondelete="CASCADE"), primary_key=True)
    product_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("products.product_id", ondelete="RESTRICT"), primary_key=True)
    quantity: Mapped[int]
    product: Mapped['Product'] = relationship()
    
# Você pode deletar todas as tabelas com esse comando
Base.metadata.drop_all(engine)

# E recria-las com esse comando
Base.metadata.create_all(engine)

In [4]:
from faker import Faker
fake = Faker()

***
### Adicionar um usuário
***

In [5]:
from sqlalchemy import insert

with Session() as session:
    for _ in range(5):
        stmt = insert(User).values(
            telegram_id=fake.pyint(),
            full_name=fake.name(),
            username=fake.user_name(),
            language_code=fake.language_code(),
            referrer_id=None,
        ).returning(User)

        result = session.execute(stmt)
        session.commit()
        data = result.scalars().first()
        print(data.full_name)

Rebecca Miller
Jacob Powers
Stephanie Gonzalez
Caitlin Good
Brittany Harris


***
### Atualizando um usuário
***

In [6]:
from sqlalchemy import update

with Session() as session:
    stmt = (
        update(User)
        .where(User.telegram_id == 3366)
        .values(referrer_id=2914, full_name="Fulano de Tal")
    ).returning(User)
    result = session.execute(stmt)
    session.commit()
    data = result.scalars().first()
    print(data.full_name, data.referrer_id)

Fulano de Tal 2914


***
### Deletando um usuário
***

In [7]:
from sqlalchemy import delete

with Session() as session:
    stmt = delete(User).where(User.telegram_id == 872).returning(User.telegram_id)
    result = session.execute(stmt)
    session.commit()
    data = result.scalar()
    print(data)

872


***
### Inserção em lote
***

In [8]:
from sqlalchemy import insert

with Session() as session:
    new_order = insert(Order).values(user_id=3366).returning(Order.order_id)
    result = session.execute(new_order)
    session.commit()
    order_id = result.scalar()
    print(order_id)

1


In [9]:
from sqlalchemy import insert, bindparam

with Session() as session:
    products = [
        dict(
            title=fake.word(),
            description=fake.sentence(),
            price=fake.pyfloat(left_digits=3, right_digits=2, positive=True)
        ) for _ in range(10)
    ]
    stmt = insert(Product).values(
        title=bindparam('title'),  # Use bindparam for batch insertion
        description=bindparam('description'),
        price=bindparam('price')
    ).returning(Product)
    result = session.execute(stmt, products)
    session.commit()
    data = result.scalars().all()
    products = [
        dict(
            product_id=product.product_id,  # Use existing product IDs
            quantity=fake.pyint()  # Generate random quantities using Faker
        ) for product in data
    ]
    print(products)

[{'product_id': 1, 'quantity': 2197}, {'product_id': 2, 'quantity': 8792}, {'product_id': 3, 'quantity': 5045}, {'product_id': 4, 'quantity': 2785}, {'product_id': 5, 'quantity': 2069}, {'product_id': 6, 'quantity': 8777}, {'product_id': 7, 'quantity': 3088}, {'product_id': 8, 'quantity': 8842}, {'product_id': 9, 'quantity': 8243}, {'product_id': 10, 'quantity': 7965}]


In [10]:
from sqlalchemy import insert

with Session() as session:
    stmt = insert(OrderProducts).values(
        order_id=order_id,  # Set the order_id
        product_id=bindparam('product_id'),  # Use bindparam for batch insertion
        quantity=bindparam('quantity')
    )
    session.execute(stmt, products)
    session.commit()

In [14]:
from sqlalchemy import select

with Session() as session:
    stmt = select(Order).where(Order.order_id == order_id)
    result = session.execute(stmt)
    order_info = result.scalars().first()
    print(f"Order ID {order_info.order_id}")
    for order_product in order_info.products:
        print("Produto:", order_product.product.title)

Order ID 1
Produto: safe
Produto: debate
Produto: chance
Produto: learn
Produto: soon
Produto: challenge
Produto: statement
Produto: support
Produto: relate
Produto: identify
